In [11]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv
import json
import numpy as np

In [12]:
load_dotenv()
opensea_api_key = os.getenv('OPENSEA_API_KEY')

# Data for Context and Analysis

### Collection Stats

In [13]:
url = "https://api.opensea.io/api/v2/collections/pudgypenguins/stats"

headers = {
    "accept": "application/json",
    "x-api-key": opensea_api_key
}

response = requests.get(url, headers=headers)

collection_stats_response_json = response.json()

collection_totals_dict = collection_stats_response_json["total"]

# I had to set this index because there is not index and it was causing an error
# I used ChatGPT to debug this issue
collection_totals_df = pd.DataFrame(collection_totals_dict, index=[0])

collection_totals_df

,volume,sales,average_price,num_owners,market_cap,floor_price,floor_price_symbol
0,345810.109458,78949,4.380171,4774,102462.604447,11.557789,ETH


## Trait Counts

In [14]:
url = "https://api.opensea.io/api/v2/traits/pudgypenguins"

headers = {
    "accept": "application/json",
    "x-api-key": opensea_api_key
}

response = requests.get(url, headers=headers)

# Formatting as json
traits_json = response.json()

# get the "counts" and not "categories" key
traits_json = traits_json.get("counts", [])

# preview
traits_json

{'Background': {'Beige': 1152,
  'Blue': 1587,
  'Mint': 1389,
  'Red': 755,
  'Tangerine': 1135,
  'Yellow': 873,
  'Pink': 710,
  'Purple': 1282,
  'On The Beach': 1,
  'Supermarket': 1,
  'Green': 1,
  'Trick Or Treating': 1,
  'Underwater': 1},
 'Skin': {'Light Gray': 1189,
  'Mint': 423,
  'Maroon': 731,
  'Normal': 2649,
  'Leopard Pink': 89,
  'Dark Gray': 1329,
  'Baby Pink': 623,
  'Cream': 646,
  'Olive Green': 705,
  'Red': 344,
  'Gold': 44,
  'Leopard Gray': 89,
  'Ice': 22,
  'Navy Blue': 4,
  'Black': 1},
 'Body': {'Vote 4 Pudgy': 203,
  'Kimono Blue': 108,
  'Scarf Blue': 237,
  'Shirt Blue': 290,
  'Fish Lover': 70,
  'Turtleneck Grey': 210,
  'Lei Pink': 168,
  'Bow Tie Blue': 154,
  'Tank Top Yellow': 193,
  'Puffer Orange': 158,
  'Huddle Shirt': 228,
  'Kimono Brown': 268,
  'Lei Assorted': 76,
  'Bronze Medal': 193,
  'Gold Medal': 58,
  'Heart': 102,
  'Turtleneck Green': 254,
  'Scarf Pink': 177,
  'Surfboard Necklace': 166,
  'Swordman': 62,
  'Christmas Sweate

#### All Traits

In [15]:
# df of all traits
# documentation: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html
# similar to how OneHotEncoder creates new column names
traits_df_normalized = pd.json_normalize(traits_json)

traits_df_normalized

,Background.Beige,Background.Blue,Background.Mint,Background.Red,Background.Tangerine,Background.Yellow,Background.Pink,Background.Purple,Background.On The Beach,Background.Supermarket,Background.Green,Background.Trick Or Treating,Background.Underwater,Skin.Light Gray,Skin.Mint,Skin.Maroon,Skin.Normal,Skin.Leopard Pink,Skin.Dark Gray,Skin.Baby Pink,Skin.Cream,Skin.Olive Green,Skin.Red,Skin.Gold,Skin.Leopard Gray,Skin.Ice,Skin.Navy Blue,Skin.Black,Body.Vote 4 Pudgy,Body.Kimono Blue,Body.Scarf Blue,Body.Shirt Blue,Body.Fish Lover,Body.Turtleneck Grey,Body.Lei Pink,Body.Bow Tie Blue,Body.Tank Top Yellow,Body.Puffer Orange,Body.Huddle Shirt,Body.Kimono Brown,...,Head.Hat Blue,Head.Fish Green,Head.Beanie Orange,Head.Wizard Hat,Head.Sombrero,Head.Beanie Pink,Head.Cowboy Hat,Head.Headband,Head.Macaroni,Head.Party Hat,Head.Biker Helmet,Head.Viking Helmet,Head.Hat Red,Head.Hatched,Head.Mohawk Purple,Head.Bowl Cut,Head.Grizzly Bear Hat,Head.Flat Cap Tan,Head.Fish Blue,Head.Fish Gold,Head.Pirate Hat,Head.Sideways Blue,Head.Top Hat,Head.Camo Helmet,Head.Rice Hat,Head.Polar Bear Hat,Head.Egg,Head.Jester's Hat,Head.Panda Hat,Head.Crown,Head.Sideways Red,Head.Egg Gold,Head.Flower Crown,Head.Fish Orange,Head.Ice Crown,Head.Hatched Gold,Head.Pineapple Suit,Head.Banana Suit,Head.Normal,Head.Shark Suit
0,1152,1587,1389,755,1135,873,710,1282,1,1,1,1,1,1189,423,731,2649,89,1329,623,646,705,344,44,89,22,4,1,203,108,237,290,70,210,168,154,193,158,228,268,...,286,117,276,186,255,90,252,357,249,196,88,196,279,53,269,89,209,274,89,22,182,89,282,248,89,89,89,176,89,58,90,45,66,44,14,9,1,1,1,1


#### Isolating Traits

In [16]:
# show all trait keys
traits_json.keys()

dict_keys(['Background', 'Skin', 'Body', 'Face', 'Head'])

In [17]:
# explode the df
traits_df_exploded =  pd.DataFrame(traits_json)
traits_df_exploded = traits_df_exploded.explode("Background")

# create new dfs for each trait, sorted by values
# if you don't do reset_index(drop=True), it appears as a series
background_trait_df = traits_df_exploded["Background"].dropna().sort_values(ascending=False).reset_index()
skin_trait_df = traits_df_exploded["Skin"].dropna().sort_values(ascending=False).reset_index()
body_trait_df = traits_df_exploded["Body"].dropna().sort_values(ascending=False).reset_index()
face_trait_df = traits_df_exploded["Face"].dropna().sort_values(ascending=False).reset_index()
head_trait_df = traits_df_exploded["Head"].dropna().sort_values(ascending=False).reset_index()

# rename columns and set indices
background_trait_df = background_trait_df.rename(columns={"index":"background", "Background":"count"}).set_index("background")
skin_trait_df = skin_trait_df.rename(columns={"index":"skin", "Skin":"count"}).set_index("skin")
body_trait_df = body_trait_df.rename(columns={"index":"body", "Body":"count"}).set_index("body")
face_trait_df = face_trait_df.rename(columns={"index":"face", "Face":"count"}).set_index("face")
head_trait_df = head_trait_df.rename(columns={"index":"head", "Head":"count"}).set_index("head")

# display trait dataframes
display(background_trait_df, skin_trait_df, body_trait_df, face_trait_df, head_trait_df)

,count
background,
Blue,1587.0
Mint,1389.0
Purple,1282.0
Beige,1152.0
Tangerine,1135.0
Yellow,873.0
Red,755.0
Pink,710.0
On The Beach,1.0


,count
skin,
Normal,2649.0
Dark Gray,1329.0
Light Gray,1189.0
Maroon,731.0
Olive Green,705.0
Cream,646.0
Baby Pink,623.0
Mint,423.0
Red,344.0


,count
body,
Shirt Blue,290.0
Kimono Brown,268.0
Shirt Red,259.0
Turtleneck Green,254.0
Scarf Green,250.0
...,...
Pineapple Suit,1.0
Banana Suit,1.0
Mirrored,1.0


,count
face,
Winking,961.0
Blushing,932.0
Normal,864.0
Circle Glasses,768.0
Cute,588.0
Monocle,496.0
Squad,445.0
Eyepatch,426.0
Cross Eyed,417.0


,count
head,
Headband,357.0
Flat Cap Blue,355.0
Bucket Hat Green,349.0
Backwards Hat Red,294.0
Hat Blue,286.0
Top Hat,282.0
Hat Red,279.0
Beanie Orange,276.0
Flat Cap Tan,274.0


# Data for Model

## NFTs

In [18]:
url = "https://api.opensea.io/api/v2/collection/pudgypenguins/nfts"

headers = {
    "accept": "application/json",
    "x-api-key": opensea_api_key
}

response = requests.get(url, headers=headers)

# Formatting as json
nfts_json = response.json()

nfts_df = pd.DataFrame(nfts_json["nfts"])

# rename "name" to "asset.name" to combine dfs
nfts_df.rename(columns={"name":"asset.name"}, inplace=True)

display(nfts_df.sample(3))

display(nfts_df.columns)

display(len(nfts_df))

,identifier,collection,contract,token_standard,asset.name,description,image_url,metadata_url,opensea_url,updated_at,is_disabled,is_nsfw
28,8828,pudgypenguins,0xbd3531da5cf5857e7cfaa92426877b022e612cf8,erc721,Pudgy Penguin #8828,A collection 8888 Cute Chubby Pudgy Penquins s...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2023-07-26T20:09:57.859296,False,False
25,8783,pudgypenguins,0xbd3531da5cf5857e7cfaa92426877b022e612cf8,erc721,Pudgy Penguin #8783,A collection 8888 Cute Chubby Pudgy Penquins s...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2024-01-20T04:07:25.913741,False,False
24,8784,pudgypenguins,0xbd3531da5cf5857e7cfaa92426877b022e612cf8,erc721,Pudgy Penguin #8784,A collection 8888 Cute Chubby Pudgy Penquins s...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2024-03-07T20:44:02.441777,True,False


Index(['identifier', 'collection', 'contract', 'token_standard', 'asset.name',
       'description', 'image_url', 'metadata_url', 'opensea_url', 'updated_at',
       'is_disabled', 'is_nsfw'],
      dtype='object')

50

## Sales Sample (50) "Events"

In [19]:

url = "https://api.opensea.io/api/v2/events/collection/pudgypenguins?event_type=sale"

headers = {
    "accept": "application/json",
    "x-api-key": opensea_api_key
}

response = requests.get(url, headers=headers)

# Formatting as json
events_json = response.json()

# get the "asset events" key and all p
events_json = events_json.get("asset_events", [])

# create df
events_df = pd.json_normalize(events_json)

# rename nft.name to asset.name
events_df.rename(columns={"nft.name":"asset.name"}, inplace=True)

display(events_df.head())

display(events_df.columns)

display(len(events_df))

,event_type,order_hash,chain,protocol_address,closing_date,quantity,seller,buyer,transaction,event_timestamp,nft.identifier,nft.collection,nft.contract,nft.token_standard,asset.name,nft.description,nft.image_url,nft.metadata_url,nft.opensea_url,nft.updated_at,nft.is_disabled,nft.is_nsfw,payment.quantity,payment.token_address,payment.decimals,payment.symbol
0,sale,,ethereum,,1714413551,1,0x29469395eaf6f95920e59f858042f0e28d98a20b,0x458dbf62e68463fe0b14c0f8dd69e695d23ca0cb,0x4829b82409ba06c652e572a852518957c11ba28d00aa...,1714413551,1266,pudgypenguins,0xbd3531da5cf5857e7cfaa92426877b022e612cf8,erc721,Pudgy Penguin #1266,A collection 8888 Cute Chubby Pudgy Penquins s...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2024-04-29T17:59:14.361366,False,False,11000000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,WETH
1,sale,,ethereum,,1714413539,1,0x29469395eaf6f95920e59f858042f0e28d98a20b,0xa69833b9fda816f1bfc79517e7932e64708df0dd,0x089d37c0aee44bf4e5916a661324417293431af23f05...,1714413539,5277,pudgypenguins,0xbd3531da5cf5857e7cfaa92426877b022e612cf8,erc721,Pudgy Penguin #5277,A collection 8888 Cute Chubby Pudgy Penquins s...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2024-04-29T17:59:02.028587,False,False,11000000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,WETH
2,sale,,ethereum,,1714405895,1,0x29469395eaf6f95920e59f858042f0e28d98a20b,0x458dbf62e68463fe0b14c0f8dd69e695d23ca0cb,0x2cb10bc67223642698383c7245763830272df45754a5...,1714405895,6062,pudgypenguins,0xbd3531da5cf5857e7cfaa92426877b022e612cf8,erc721,Pudgy Penguin #6062,A collection 8888 Cute Chubby Pudgy Penquins s...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2024-04-29T15:51:38.163165,False,False,11060000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,WETH
3,sale,,ethereum,,1714405895,1,0x29469395eaf6f95920e59f858042f0e28d98a20b,0x458dbf62e68463fe0b14c0f8dd69e695d23ca0cb,0xe3155e33aabc5dc62b2f94a90447b2ba606721f71224...,1714405895,8852,pudgypenguins,0xbd3531da5cf5857e7cfaa92426877b022e612cf8,erc721,Pudgy Penguin #8852,A collection 8888 Cute Chubby Pudgy Penquins s...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2024-04-29T15:51:38.153032,False,False,11060000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,WETH
4,sale,,ethereum,,1714405883,1,0x29469395eaf6f95920e59f858042f0e28d98a20b,0x458dbf62e68463fe0b14c0f8dd69e695d23ca0cb,0x5769f5a98ca19e6106685a135105ccfd40bc42017d2e...,1714405883,8390,pudgypenguins,0xbd3531da5cf5857e7cfaa92426877b022e612cf8,erc721,Pudgy Penguin #8390,A collection 8888 Cute Chubby Pudgy Penquins s...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2024-04-29T15:51:26.690060,False,False,11060000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,WETH


Index(['event_type', 'order_hash', 'chain', 'protocol_address', 'closing_date',
       'quantity', 'seller', 'buyer', 'transaction', 'event_timestamp',
       'nft.identifier', 'nft.collection', 'nft.contract',
       'nft.token_standard', 'asset.name', 'nft.description', 'nft.image_url',
       'nft.metadata_url', 'nft.opensea_url', 'nft.updated_at',
       'nft.is_disabled', 'nft.is_nsfw', 'payment.quantity',
       'payment.token_address', 'payment.decimals', 'payment.symbol'],
      dtype='object')

50

### Individual NFT Info

#### query parameters from https://docs.opensea.io/reference/get_nft

In [20]:
# PATH PARAMETERS

# note: nft contract is the same for the whole collection

# identify nft contract (public blockchain address)
# this creates an array
protocol_address = events_df["nft.contract"].unique()

# eliminate the null by getting the max of the array
protocol_address = np.max(protocol_address)

# find unique values for chain (currency)
chain = events_df["chain"].unique().tolist()

# write value without list
chain = chain[0]

# token ID in a list for iteration
token_id_list = events_df["nft.identifier"].to_list()

In [21]:
# parsing features function

def get_features (nft):
    features = {}
    features["Background"] = nft["traits"][0]["value"]
    features["Skin"] = nft["traits"][1]["value"]
    features["Face"] = nft["traits"][2]["value"]
    features["Head"] = nft["traits"][3]["value"]
    features["Body"] = nft["traits"][4]["value"]
    features["Rarity"] = nft["rarity"]["rank"]
    
    return features

In [22]:
nfts_dict = {}
token_id = token_id_list[0]

for token_id in token_id_list:
    url = f"https://api.opensea.io/api/v2/chain/{chain}/contract/{protocol_address}/nfts/{token_id}"
    headers = {
        "accept": "application/json",
        "x-api-key": "5b828d394dd74a99ad2f9142157215d2"
    }

    response = requests.get(url, headers=headers)

    # Formatting as json
    nfts_json = response.json()
    
    nfts_dict[token_id] = get_features(nfts_json["nft"])

In [23]:
nfts_dict

nfts_df = pd.DataFrame(nfts_dict).transpose()

nfts_df

,Background,Skin,Face,Head,Body,Rarity
1266,Blue,Normal,Turtleneck Grey,Eyepatch,Camo Helmet,7978
5277,Light Gray,Beard,Pink,Turtleneck Green,Cowboy Hat,3623
6062,Yellow,Cream,Clout Goggles,Camo Helmet,Ice Coat,1231
8852,Fish Lover,Backwards Hat Red,Purple,Red,Monocle,917
8390,Blue,Santa Hat,Normal,Cute,Lei Purple,8016
8581,Beige,Winking,Maroon,Fish Lover,Macaroni,3042
2034,Red,Tank Top Yellow,Olive Green,Hat Red,Monocle,3068
5033,Mint,Bow Tie Blue,Baby Pink,Afro With Pick,Circle Glasses,4787
5783,Blue,Maroon,Puffer Green,Mustache,Camo Helmet,1753
5456,Light Gray,Durag Blue,Yellow,Aviator,Lab Coat,4046


In [24]:
combined_events_df = events_df[["nft.identifier", "closing_date", "payment.quantity"]]

combined_events_df.set_index("nft.identifier", inplace=True)

combined_events_df

,closing_date,payment.quantity
nft.identifier,,
1266,1714413551,11000000000000000000
5277,1714413539,11000000000000000000
6062,1714405895,11060000000000000000
8852,1714405895,11060000000000000000
8390,1714405883,11060000000000000000
8581,1714405883,11060000000000000000
2034,1714405031,11060000000000000000
5033,1714405031,11060000000000000000
5783,1714405019,11060000000000000000


In [25]:
combined_df = combined_events_df.merge(nfts_df, how="outer", right_index=True, left_index=True)

combined_df

,closing_date,payment.quantity,Background,Skin,Face,Head,Body,Rarity
nft.identifier,,,,,,,,
1049,1714367927,11910000000000000000,Light Gray,Normal,Tangerine,Hat Blue,Puffer Green,6938
1167,1714367939,11910000000000000000,Blue,Cute,Durag Blue,Baby Pink,Hawaiian Shirt,4663
1266,1714413551,11000000000000000000,Blue,Normal,Turtleneck Grey,Eyepatch,Camo Helmet,7978
1266,1714404971,11060000000000000000,Blue,Normal,Turtleneck Grey,Eyepatch,Camo Helmet,7978
2034,1714405031,11060000000000000000,Red,Tank Top Yellow,Olive Green,Hat Red,Monocle,3068
2773,1714374851,11500000000000000000,Beige,Maroon,Cute,Beanie Orange,Scarf Green,5598
2773,1714374707,11080000000000000000,Beige,Maroon,Cute,Beanie Orange,Scarf Green,5598
2773,1714370747,11000000000000000000,Beige,Maroon,Cute,Beanie Orange,Scarf Green,5598
3372,1714390271,11060000000000000000,Light Gray,Pink,Blushing,Kimono Brown,Hat Blue,7130


In [26]:
combined_df.to_csv("mariamaria4.25.2024")

In [1]:
def convert_usd_to_eth(usd_amount):
    # API URL for getting the latest ETH price in USD
    url = "https://api.coingecko.com/api/v3/simple/price?ids=ethereum&vs_currencies=usd"
    
    try:
        response = requests.get(url)
        data = response.json()
        
        # Getting the current price of 1 ETH in USD
        eth_price = data['ethereum']['usd']
        
        # Calculating the amount in ETH
        eth_amount = usd_amount / eth_price
        
        return eth_amount
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Example: Convert 100 USD to ETH
usd_amount = 100
eth_amount = convert_usd_to_eth(usd_amount)
eth_amount

An error occurred: name 'requests' is not defined
